# Ranking Strategies Notebook
This notebook normalized cumulative, reliability, and decay metrics and ranks strategies highest to lowest based on these metrics.

## Import Libraries

In [1]:
from ipywidgets import interact, interact_manual
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import  util
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from EquityHedging.reporting import formatter as fmt
from EquityHedging.analytics import hedge_metrics as hm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go


## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [2]:
equity_bmk = 'M1WD'
include_fi = False
weighted = [True, False]
strat_drop_list = ['99%/90% Put Spread', 'Vortex']
new_strat = True
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

##  Get Notional Weights 

In [3]:
#notional_weights = dm.get_notional_weights(returns['Monthly'])
notional_weights = [19.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.25, 1.0]
df_weights = get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

,M1WD,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var
Notional Weights (Billions),$19.00,$1.00,$1.00,$1.00,$1.00,$1.00,$0.25,$1.00
Percentage Weights,100.00%,5.26%,5.26%,5.26%,5.26%,5.26%,1.32%,5.26%
Strategy Weights,0.00%,16.00%,16.00%,16.00%,16.00%,16.00%,4.00%,16.00%


## Get Data

In [4]:
data = summary.get_normalized_hedge_metrics(returns, equity_bmk, notional_weights, weighted_hedge = True)
data['Normalized Data']

,Benefit,Downside Reliability,Upside Reliability,Convexity,Cost,Decay
Down Var,0.447231,0.464568,1.000000,0.710750,0.086945,0.337719
VOLA,0.000000,1.000000,0.257301,1.000000,0.302052,0.828947
Dynamic Put Spread,0.155304,0.845703,0.000000,0.035714,0.535959,0.293860
VRR,1.000000,0.027477,0.699599,0.310656,0.000000,0.000000
GW Dispersion,0.243241,0.598862,0.746761,0.119773,0.668714,1.000000
Corr Hedge,0.374030,0.127817,0.856815,0.844444,0.569583,0.877193
Def Var,0.422692,0.000000,0.618485,0.000000,1.000000,0.767544
Weighted Hedges,0.007034,0.888764,0.567331,0.186045,0.919781,0.951754


In [5]:
df_norm = data['Normalized Data']
df_hm = data['Hedge Metrics']

## Display Data

In [6]:
@interact
def display_hedge_metrics_or_normal(type = ['Hedge Metrics','Normalized Data']):
    if type == 'Hedge Metrics':
        return fmt.format_hm_to_normalize(df_hm)    
    if type == 'Normalized Data':
        return fmt.format_normalized_data(df_norm)


interactive(children=(Dropdown(description='type', options=('Hedge Metrics', 'Normalized Data'), value='Hedge …

## Creates a List of Symbols 

In [7]:
df_normal = df_norm.transpose()
symbols = util.get_symbols(df_normal, weighted_hedge = True)


## Graph Normalized Data and Ranking Each Strategy

In [10]:
#transpose normal data here instead of in method
index = list(df_normal.index)
fig = go.Figure()
#loops through eaach column and plots each metric for the given strategy
for col in df_normal.columns:
    fig.add_trace(go.Scatter(
        x=index,
        y=df_normal[col],
        marker=dict(size = 20),
        marker_symbol = symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(title="",
                xaxis_title="Hedge Metric",
                yaxis_title="Higher the Better")
fig.show()